# 3.8 推論の実施

- 本ファイルでは、学習させたPSPNetでセマンティックセグメンテーションを行います。


# 学習目標


1.	セマンティックセグメンテーションの推論を実装できるようになる


# 事前準備

- 学習させた重みパラメータ「pspnet50_30.pth」をフォルダ「weights」に用意する。

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import torch.nn.functional as F
import torch

# ファイルパスリストを用意

In [ ]:
import sys

In [ ]:
sys.path.append('../semantic_segmentation/')

In [ ]:
from utils.numpy_dataloader import make_datapath_list  #, DataTransform


# ファイルパスリスト作成
rootpath = "../semantic_segmentation/data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(
    rootpath=rootpath)

# 後ほどアノテーション画像のみを使用する


# ネットワークを用意

In [ ]:
#####################################



# from utils.pspnet import PSPNet
# from torchsummary import summary
# net = PSPNet(n_classes=2)
# # net.to("cuda")
# # net150 = PSPNet(n_classes=150)
# # net150.to("cuda")

# # 学習済みパラメータをロード
# state_dict = torch.load("../semantic_segmentation/weights/pspnet50_500_1111_img200_batch16_net.pth",
#                         map_location={'cuda:0': 'cpu'})
# net.load_state_dict(state_dict)

# print('ネットワーク設定完了：学習済みの重みをロードしました')


###########################################

In [ ]:
####################
# net = PSPNet(n_classes=2)
####################

In [ ]:
# net150 = PSPNet(n_classes=150)

In [ ]:
# net150.to("cuda")

In [ ]:
###################
# net.to("cuda")
####################

In [ ]:
# summary(net,(1,200,200))

In [ ]:
from utils.pspnet import PSPNet
from torchsummary import summary
# net = PSPNet(n_classes=2)
net = PSPNet(n_classes=2)

# 学習済みパラメータをロード
state_dict = torch.load("../semantic_segmentation/weights/np_array_weight/300_net.pth",
                        map_location={'cuda:0': 'cpu'})
#/content/drive/MyDrive/3_semantic_segmentation/weights/pspnet50_30_1231.pth このweightの出力は150　これが原因
net.load_state_dict(state_dict)

print('ネットワーク設定完了：学習済みの重みをロードしました')


# 推論実行

In [ ]:
# image_file_path = "../semantic_segmentation/data/VOCdevkit/VOC2012/gray_cat_JPEGImages/2007_004856.jpg"
# img = Image.open(image_file_path)   # [高さ][幅][色RGB]
# img_width, img_height = img.size
# plt.imshow(img)
# plt.show()

# # 2. 前処理クラスの作成
# # color_mean = (0.485, 0.456, 0.406)
# # color_std = (0.229, 0.224, 0.225)
# color_mean = 0.5 #1ch
# color_std = 0.2
# transform = DataTransform(
#     input_size=200, color_mean=color_mean, color_std=color_std)

# # 3. 前処理
# # 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
# anno_file_path = val_anno_list[0]
# anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
# p_palette = anno_class_img.getpalette()
# phase = "val"
# img, anno_class_img = transform(phase, img, anno_class_img)


In [ ]:
# img.cuda().type()


In [ ]:
def resize(data, size):
    """
    sizeは、自由　　　　　　
    今はy ,xは同じサイズだが、違うサイズにしたければ、タプルでsizeを入力するとよい
    入力データ：（y, x, 2 or 3）
    出力：（size ,size, 2 or 3）
    """
#     cut_data = np.swapaxes(data, 0, 2)
#     cut_data = np.swapaxes(cut_data, 0, 1)
#     print(cut_data.shape)
    cut_data = data[None,:,:].astype(np.float32)
    cut_data = torch.from_numpy(cut_data)
    cut_data = cut_data.unsqueeze(0)
    resize_data = F.interpolate(cut_data, (size, size), mode='bilinear', align_corners=False)
    resize_data = np.squeeze(resize_data.detach().numpy())

#     resize_data_ = np.swapaxes(resize_data, 0, 1)
#     resize_data_ = np.swapaxes(resize_data_, 1, 2)
    return resize_data


In [ ]:
i = 5
arr_file_path = "/home/filament/Downloads/semantic_segmentation/data/VOCdevkit/VOC2012/filament_taurus_datasets_grey_arcnorm_array/taurus_%s.npy"%i
image_file_path = "/home/filament/Desktop/taurus_img/taurus_%s.jpg"%i
# img = Image.open(image_file_path)   # [高さ][幅][色RGB]
arr = np.load(arr_file_path)

In [ ]:
for i in range(1,45):
    fig,ax = plt.subplots(1,3)
    # arr_file_path = "/home/filament/Downloads/semantic_segmentation/data/VOCdevkit/VOC2012/filament_taurus_datasets_grey_arcnorm_array/taurus_%s.npy"%i
    arr_file_path = "/home/filament/Downloads/semantic_segmentation/data/VOCdevkit/VOC2012/fil_norm_data/taurus_%s.npy"%i
    image_file_path = "/home/filament/Desktop/taurus_img/taurus_%s.jpg"%i
    # img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    arr = np.load(arr_file_path)
    arr_width, arr_height = arr.shape[1],arr.shape[0]
    # arr = np.array(img)
    ax[0].imshow(arr)
    

    arr = torch.from_numpy(resize(arr,475))
    anno_file_path = val_anno_list[0]
    anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
    p_palette = anno_class_img.getpalette()

    net.eval()
    x = arr.unsqueeze(0)# ミニバッチ化：torch.Size([1, 3, 475, 475])
    x = x.unsqueeze(0)
    outputs = net(x)
    y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])

    y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
    y = np.argmax(y, axis=0)
    anno_class_img = Image.fromarray(np.uint8(y), mode="P")
    anno_class_img = anno_class_img.resize((arr_height, arr_width), Image.NEAREST)
    anno_class_img.putpalette(p_palette)
    ax[1].imshow(anno_class_img)
    

    trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
    anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

    for x in range(arr_width):
        for y in range(arr_height):
            # 推論結果画像のピクセルデータを取得
            pixel = anno_class_img.getpixel((x, y))
            r, g, b, a = pixel

            # (0, 0, 0)の背景ならそのままにして透過させる
            if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                continue
            else:
                # それ以外の色は用意した画像にピクセルを書き込む
                trans_img.putpixel((x, y), (r, g, b, 150))
                # 150は透過度の大きさを指定している

    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    result = Image.alpha_composite(img.convert('RGBA'), trans_img)
    ax[2].imshow(result)
    plt.show()
    plt.clf()

In [ ]:
num_y = y.detach().numpy()

In [ ]:
arg_y = np.argmax(num_y[0],axis=0)

In [ ]:
fig,ax = plt.subplots(2,3,figsize=(16,16))
ax[0,0].imshow(arr)
ax[0,1].imshow(outputs[0][0][1].detach().numpy())
ax[0,2].imshow(outputs[0][0][0].detach().numpy())
ax[1,0].imshow(arg_y)
ax[1,1].imshow(anno_class_img)
ax[1,2].imshow(result)

In [ ]:
plt.imshow(arg_y)

In [ ]:
# anno_class_img = Image.fromarray(np.uint8(arg_y), mode="P")
# anno_class_img = anno_class_img.resize((arr_width, arr_height), Image.NEAREST)
# anno_class_img.putpalette(p_palette)
# plt.imshow(anno_class_img)
# plt.show()
trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

for x in range(arr_width):
    for y in range(arr_height):
        # 推論結果画像のピクセルデータを取得
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel

        # (0, 0, 0)の背景ならそのままにして透過させる
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # それ以外の色は用意した画像にピクセルを書き込む
            trans_img.putpixel((x, y), (r, g, b, 150))
            # 150は透過度の大きさを指定している

img = Image.open(image_file_path)   # [高さ][幅][色RGB]
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()
plt.clf()

In [ ]:
plt.imshow(outputs[0][0][1].detach().numpy())

In [ ]:
tensor = torch.from_numpy(arr)
tensor.shape

In [ ]:
tensor_2 = tensor.unsqueeze(0)

In [ ]:
tensor_2.shape

In [ ]:
arr.size

In [ ]:
720*1079*3

In [ ]:
# 1. 元画像の表示
# image_file_path = "../semantic_segmentation/data/VOCdevkit/VOC2012/JPEGImages/2007_000392.jpg"
image_file_path = "../neko_jpg/inu_1.jpg"
img = Image.open(image_file_path)   # [高さ][幅][色RGB]
img_width, img_height = img.size
plt.imshow(img)
plt.show()

# 2. 前処理クラスの作成
# color_mean = (0.485, 0.456, 0.406)
# color_std = (0.229, 0.224, 0.225)
color_mean = 0.5 #1ch
color_std = 0.2
transform = DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std)

# 3. 前処理
# 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
anno_file_path = val_anno_list[0]
anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
p_palette = anno_class_img.getpalette()
phase = "val"
img, anno_class_img = transform(phase, img, anno_class_img)
####################
# img = img.cuda()
# anno_class_img = anno_class_img.cuda()
####################

# 4. PSPNetで推論する
net.eval()
x = img.unsqueeze(0) # ミニバッチ化：torch.Size([1, 3, 475, 475])
outputs = net(x)
y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


#################
# y = y.cpu()
#################

# 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
y = np.argmax(y, axis=0)
anno_class_img = Image.fromarray(np.uint8(y), mode="P")
anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
anno_class_img.putpalette(p_palette)
plt.imshow(anno_class_img)
plt.show()


# 6. 画像を透過させて重ねる
trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

for x in range(img_width):
    for y in range(img_height):
        # 推論結果画像のピクセルデータを取得
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel

        # (0, 0, 0)の背景ならそのままにして透過させる
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # それ以外の色は用意した画像にピクセルを書き込む
            trans_img.putpixel((x, y), (r, g, b, 150))
            # 150は透過度の大きさを指定している

img = Image.open(image_file_path)   # [高さ][幅][色RGB]
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()
plt.clf()


In [ ]:
y.shape

In [ ]:
y[0]

In [ ]:
# 1. 元画像の表示
image_file_path = "../itijitekinayatu/orionKL_1 (1).jpg"
img = Image.open(image_file_path)   # [高さ][幅][色RGB]
img_width, img_height = img.size
plt.imshow(img)
plt.show()


# 2. 前処理クラスの作成
# color_mean = (0.485, 0.456, 0.406)
# color_std = (0.229, 0.224, 0.225)
color_mean = 0.5 #1ch
color_std = 0.2
transform = DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std)

# 3. 前処理
# 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
anno_file_path = val_anno_list[0]
anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
p_palette = anno_class_img.getpalette()
phase = "val"
img, anno_class_img = transform(phase, img, anno_class_img)
####################
# img = img.cuda()
# anno_class_img = anno_class_img.cuda()
####################

# 4. PSPNetで推論する
net.eval()
x = img.unsqueeze(0) # ミニバッチ化：torch.Size([1, 3, 475, 475])
outputs = net(x)
y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


#################
# y = y.cpu()
#################

# 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
y = np.argmax(y, axis=0)
anno_class_img = Image.fromarray(np.uint8(y), mode="P")
anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
anno_class_img.putpalette(p_palette)
plt.imshow(anno_class_img)
plt.show()


# 6. 画像を透過させて重ねる
trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

for x in range(img_width):
    for y in range(img_height):
        # 推論結果画像のピクセルデータを取得
        pixel = anno_class_img.getpixel((x, y))
        r, g, b, a = pixel

        # (0, 0, 0)の背景ならそのままにして透過させる
        if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
            continue
        else:
            # それ以外の色は用意した画像にピクセルを書き込む
            trans_img.putpixel((x, y), (r, g, b, 150))
            # 150は透過度の大きさを指定している

img = Image.open(image_file_path)   # [高さ][幅][色RGB]
result = Image.alpha_composite(img.convert('RGBA'), trans_img)
plt.imshow(result)
plt.show()
plt.clf()


In [ ]:
image_file_path = "../semantic_segmentation/data/VOCdevkit/VOC2012/filament_taurus_datasets_grey/taurus_"+str(1)+".jpg"

In [ ]:
image_file_path.split("/")[6].split(".")[0]

In [ ]:
for i in range(15):
    # 1. 元画像の表示
    image_file_path = "../semantic_segmentation/data/VOCdevkit/VOC2012/filament_taurus_datasets_grey/taurus_"+str(i+1)+".jpg"
    fname = image_file_path.split("/")[6].split(".")[0]
    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    img_width, img_height = img.size
    plt.imshow(img)
    plt.imsave("../semantic_segmentation/inference/original/" + fname + ".jpg",img)
    plt.show()


    # 2. 前処理クラスの作成
    # color_mean = (0.485, 0.456, 0.406)
    # color_std = (0.229, 0.224, 0.225)
    color_mean = 0.5 #1ch
    color_std = 0.2
    transform = DataTransform(
        input_size=475, color_mean=color_mean, color_std=color_std)

    # 3. 前処理
    # 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
    anno_file_path = val_anno_list[0]
    anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
    p_palette = anno_class_img.getpalette()
    phase = "val"
    img, anno_class_img = transform(phase, img, anno_class_img)
    ####################
    # img = img.cuda()
    # anno_class_img = anno_class_img.cuda()
    ####################

    # 4. PSPNetで推論する
    net.eval()
    x = img.unsqueeze(0) # ミニバッチ化：torch.Size([1, 3, 475, 475])
    outputs = net(x)
    y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


    #################
    # y = y.cpu()
    #################

    # 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
    y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
    y = np.argmax(y, axis=0)
    anno_class_img = Image.fromarray(np.uint8(y), mode="P")
    anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
    anno_class_img.putpalette(p_palette)
    plt.imshow(anno_class_img)
    anno_class_img.save("../semantic_segmentation/inference/inference/" + fname + ".png")
    plt.show()


    # 6. 画像を透過させて重ねる
    trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
    anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

    for x in range(img_width):
        for y in range(img_height):
            # 推論結果画像のピクセルデータを取得
            pixel = anno_class_img.getpixel((x, y))
            r, g, b, a = pixel

            # (0, 0, 0)の背景ならそのままにして透過させる
            if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                continue
            else:
                # それ以外の色は用意した画像にピクセルを書き込む
                trans_img.putpixel((x, y), (r, g, b, 150))
                # 150は透過度の大きさを指定している

    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    result = Image.alpha_composite(img.convert('RGBA'), trans_img)
    plt.imshow(result)
    result.save("../semantic_segmentation/inference/orginal_inference/" + fname + ".png")
    plt.show()
    plt.clf()


In [ ]:
fpath = glob.glob("../semantic_segmentation/test/test_val/*.jpg")

In [ ]:
fpath[9].split("/")[4].split(".")[0]

In [ ]:
fpath = glob.glob("../semantic_segmentation/test/test_val/*.jpg")


for path in fpath:
    # 1. 元画像の表示
    image_file_path = path
    fname = image_file_path.split("/")[4].split(".")[0]
    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    img_width, img_height = img.size
    plt.imshow(img)
    plt.imsave("../semantic_segmentation/inference/original/" + fname + ".jpg",img)
    plt.show()


    # 2. 前処理クラスの作成
    # color_mean = (0.485, 0.456, 0.406)
    # color_std = (0.229, 0.224, 0.225)
    color_mean = 0.5 #1ch
    color_std = 0.2
    transform = DataTransform(
        input_size=475, color_mean=color_mean, color_std=color_std)

    # 3. 前処理
    # 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
    anno_file_path = val_anno_list[0]
    anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
    p_palette = anno_class_img.getpalette()
    phase = "val"
    img, anno_class_img = transform(phase, img, anno_class_img)
    ####################
    # img = img.cuda()
    # anno_class_img = anno_class_img.cuda()
    ####################

    # 4. PSPNetで推論する
    net.eval()
    x = img.unsqueeze(0) # ミニバッチ化：torch.Size([1, 3, 475, 475])
    outputs = net(x)
    y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


    #################
    # y = y.cpu()
    #################

    # 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
    y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
    y = np.argmax(y, axis=0)
    anno_class_img = Image.fromarray(np.uint8(y), mode="P")
    anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
    anno_class_img.putpalette(p_palette)
    plt.imshow(anno_class_img)
    anno_class_img.save("../semantic_segmentation/inference/inference/" + fname + ".png")
    plt.show()


    # 6. 画像を透過させて重ねる
    trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
    anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

    for x in range(img_width):
        for y in range(img_height):
            # 推論結果画像のピクセルデータを取得
            pixel = anno_class_img.getpixel((x, y))
            r, g, b, a = pixel

            # (0, 0, 0)の背景ならそのままにして透過させる
            if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                continue
            else:
                # それ以外の色は用意した画像にピクセルを書き込む
                trans_img.putpixel((x, y), (r, g, b, 150))
                # 150は透過度の大きさを指定している

    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    result = Image.alpha_composite(img.convert('RGBA'), trans_img)
    plt.imshow(result)
    result.save("../semantic_segmentation/inference/orginal_inference/" + fname + ".png")
    plt.show()
    plt.clf()


In [ ]:
for i in range(15,19):
    # 1. 元画像の表示
    image_file_path = "../semantic_segmentation/data/VOCdevkit/VOC2012/filament_taurus_datasets_grey/taurus_"+str(i+1)+".jpg"
    fname = image_file_path.split("/")[6].split(".")[0]
    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    img_width, img_height = img.size
    plt.imshow(img)
    plt.imsave("../semantic_segmentation/inference/original/" + fname + ".jpg",img)
    plt.show()


    # 2. 前処理クラスの作成
    # color_mean = (0.485, 0.456, 0.406)
    # color_std = (0.229, 0.224, 0.225)
    color_mean = 0.5 #1ch
    color_std = 0.2
    transform = DataTransform(
        input_size=475, color_mean=color_mean, color_std=color_std)

    # 3. 前処理
    # 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
    anno_file_path = val_anno_list[0]
    anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
    p_palette = anno_class_img.getpalette()
    phase = "val"
    img, anno_class_img = transform(phase, img, anno_class_img)
    ####################
    # img = img.cuda()
    # anno_class_img = anno_class_img.cuda()
    ####################

    # 4. PSPNetで推論する
    net.eval()
    x = img.unsqueeze(0) # ミニバッチ化：torch.Size([1, 3, 475, 475])
    outputs = net(x)
    y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])


    #################
    # y = y.cpu()
    #################

    # 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
    y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
    y = np.argmax(y, axis=0)
    anno_class_img = Image.fromarray(np.uint8(y), mode="P")
    anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
    anno_class_img.putpalette(p_palette)
    plt.imshow(anno_class_img)
    anno_class_img.save("../semantic_segmentation/inference/inference/" + fname + ".png")
    plt.show()


    # 6. 画像を透過させて重ねる
    trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
    anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

    for x in range(img_width):
        for y in range(img_height):
            # 推論結果画像のピクセルデータを取得
            pixel = anno_class_img.getpixel((x, y))
            r, g, b, a = pixel

            # (0, 0, 0)の背景ならそのままにして透過させる
            if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
                continue
            else:
                # それ以外の色は用意した画像にピクセルを書き込む
                trans_img.putpixel((x, y), (r, g, b, 150))
                # 150は透過度の大きさを指定している

    img = Image.open(image_file_path)   # [高さ][幅][色RGB]
    result = Image.alpha_composite(img.convert('RGBA'), trans_img)
    plt.imshow(result)
    result.save("../semantic_segmentation/inference/orginal_inference/" + fname + ".png")
    plt.show()
    plt.clf()


In [ ]:
def Segmentation(path):
  # 1. 元画像の表示
  image_file_path = path
  fname = path.split("/")[6].split(".")[0]
  img = Image.open(image_file_path)   # [高さ][幅][色RGB]
  img_width, img_height = img.size
  plt.imshow(img)
  img.save("../semantic_segmentation/inference/original/" + fname + ".jpg")
  plt.show()

  # 2. 前処理クラスの作成
  # color_mean = (0.485, 0.456, 0.406)
  # color_std = (0.229, 0.224, 0.225)
  color_mean = 0.5 #1ch
  color_std = 0.2
  transform = DataTransform(
      input_size=475, color_mean=color_mean, color_std=color_std)

  # 3. 前処理
  # 適当なアノテーション画像を用意し、さらにカラーパレットの情報を抜き出す
  anno_file_path = val_anno_list[0]
  anno_class_img = Image.open(anno_file_path)   # [高さ][幅]
  p_palette = anno_class_img.getpalette()
  phase = "val"
  img, anno_class_img = transform(phase, img, anno_class_img)
  #################
  # img = img.cuda()
  # anno_class_img = anno_class_img.cuda()
  #################


  # 4. PSPNetで推論する
  net.eval()
  x = img.unsqueeze(0)  # ミニバッチ化：torch.Size([1, 3, 475, 475])
  outputs = net(x)
  y = outputs[0]  # AuxLoss側は無視 yのサイズはtorch.Size([1, 21, 475, 475])
  ###################
  # y = y.cpu()
  ###################



  # 5. PSPNetの出力から最大クラスを求め、カラーパレット形式にし、画像サイズを元に戻す
  y = y[0].detach().numpy()  # y：torch.Size([1, 21, 475, 475])
  y = np.argmax(y, axis=0)
  anno_class_img = Image.fromarray(np.uint8(y), mode="P")
  anno_class_img = anno_class_img.resize((img_width, img_height), Image.NEAREST)
  anno_class_img.putpalette(p_palette)
  plt.imshow(anno_class_img)
  anno_class_img.save("../semantic_segmentation/inference/inference/" + fname + ".png")
  plt.show()


  # 6. 画像を透過させて重ねる
  trans_img = Image.new('RGBA', anno_class_img.size, (0, 0, 0, 0))
  anno_class_img = anno_class_img.convert('RGBA')  # カラーパレット形式をRGBAに変換

  for x in range(img_width):
      for y in range(img_height):
          # 推論結果画像のピクセルデータを取得
          pixel = anno_class_img.getpixel((x, y))
          r, g, b, a = pixel

          # (0, 0, 0)の背景ならそのままにして透過させる
          if pixel[0] == 0 and pixel[1] == 0 and pixel[2] == 0:
              continue
          else:
              # それ以外の色は用意した画像にピクセルを書き込む
              trans_img.putpixel((x, y), (r, g, b, 150))
              # 150は透過度の大きさを指定している

  img = Image.open(image_file_path)   # [高さ][幅][色RGB]
  result = Image.alpha_composite(img.convert('RGBA'), trans_img)
  plt.imshow(result)
  result.save("../semantic_segmentation/inference/orginal_inference/" + fname + ".png")
  plt.show()
  plt.clf()

In [ ]:
val_img_list

In [ ]:
train_img_list

In [ ]:
len(val_img_list)

In [ ]:
len(train_img_list)

In [ ]:
for i in val_img_list:
  Segmentation(i)

以上

In [ ]:
for i in train_img_list:
  Segmentation(i)